In [20]:
import requests, json
from pprint import pprint

In [21]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'chips': [...],
  'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_config': {...},
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 4821950}


In [22]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'birth_date': '1995-09-15',
 'bonus': 10,
 'bps': 555,
 'can_select': True,
 'can_transact': True,
 'chance_of_playing_next_round': None,
 'chance_of_playing_this_round': None,
 'clean_sheets': 13,
 'clean_sheets_per_90': 0.34,
 'clearances_blocks_interceptions': 0,
 'code': 154561,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': 0,
 'cost_change_start_fall': 0,
 'creativity': '10.7',
 'creativity_rank': 342,
 'creativity_rank_type': 11,
 'defensive_contribution': 0,
 'defensive_contribution_per_90': 0.0,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 1,
 'ep_next': '4.5',
 'ep_this': None,
 'event_points': 0,
 'expected_assists': '0.03',
 'expected_assists_per_90': 0.0,
 'expected_goal_involvements': '0.03',
 'expected_goal_involvements_per_90': 0.0,
 'expected_goals': '0.00',
 'expected_goals_co

In [23]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [24]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Raya,1,1
1,2,Arrizabalaga,1,1
2,3,Hein,1,1
3,4,Setford,1,1
4,5,Gabriel,1,2


In [25]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1320,1325,1350,1350,1290,1300,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1125,1250,1110,1200,1140,1300,2
2,90,0,None,3,0,Burnley,0,0,0,BUR,2,None,False,0,1050,1050,1050,1050,1050,1050,43
3,91,0,None,4,0,Bournemouth,0,0,0,BOU,3,None,False,0,1150,1180,1100,1160,1200,1200,127
4,94,0,None,5,0,Brentford,0,0,0,BRE,3,None,False,0,1120,1185,1080,1080,1160,1290,130


In [26]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],79
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],223
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],304
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],71


In [27]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,David,Raya Martín,Arsenal
1,Kepa,Arrizabalaga Revuelta,Arsenal
2,Karl,Hein,Arsenal
3,Tommy,Setford,Arsenal
4,Gabriel,dos Santos Magalhães,Arsenal


In [28]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,David,Raya Martín,Arsenal,Goalkeeper
1,Kepa,Arrizabalaga Revuelta,Arsenal,Goalkeeper
2,Karl,Hein,Arsenal,Goalkeeper
3,Tommy,Setford,Arsenal,Goalkeeper
4,Gabriel,dos Santos Magalhães,Arsenal,Defender


Gameweek points from the current **season**
https://fantasy.premierleague.com/api/entry/{TID}/event/{GW}/picks/
https://fantasy.premierleague.com/api/element-summary/{EID}/

Option 2

In [29]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)


{'fixtures': [...], 'history': [], 'history_past': [...]}


In [30]:
if r.get('history_past'):
    pprint(r['history_past'][0])  # First gameweek
else:
    print("No gameweek history found.")

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'clearances_blocks_interceptions': 0,
 'creativity': '0.0',
 'defensive_contribution': 0,
 'element_code': 551221,
 'end_cost': 45,
 'expected_assists': '0.00',
 'expected_goal_involvements': '0.00',
 'expected_goals': '0.00',
 'expected_goals_conceded': '0.00',
 'goals_conceded': 0,
 'goals_scored': 0,
 'ict_index': '0.0',
 'influence': '0.0',
 'minutes': 0,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'recoveries': 0,
 'red_cards': 0,
 'saves': 0,
 'season_name': '2024/25',
 'start_cost': 45,
 'starts': 0,
 'tackles': 0,
 'threat': '0.0',
 'total_points': 0,
 'yellow_cards': 0}


In [31]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''

    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()

    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history_past'])

    return df


# show player #4's gameweek history
get_gameweek_history(1)[ # Raya
    [
        'season_name',
        'total_points',
        'minutes',
        'clean_sheets',
        'goals_conceded'
    ]
].head()

,season_name,total_points,minutes,clean_sheets,goals_conceded
0,2021/22,95,2160,8,27
1,2022/23,166,3420,12,46
2,2023/24,135,2880,16,24
3,2024/25,142,3420,13,34


In [32]:
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,birth_date,has_temporary_code,opta_code,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,defensive_contribution_per_90
0,True,True,NaN,None,154561,0,0,0,0,0,1,4.5,None,0,David,0.0,1,False,,None,55,154561.jpg,3.7,False,Raya Martín,17.8,False,None,a,1,3,142,0,0,0,0,0.0,25.8,Raya,200.0,2024-07-04,1995-09-15,False,p154561,3420,0,0,13,34,0,0,0,3,0,86,10,555,755.4,10.7,0.0,76.7,0,0,0,0,38,0.00,0.03,0.03,35.03,37,8,342,11,640,79,178,8,NaN,,NaN,,NaN,,0.0,2.26,0.00,0.00,0.92,0.89,129,4,481,79,68,11,22,4,1.00,0.34,0.00
1,True,True,NaN,None,109745,0,0,0,0,0,1,2.8,None,0,Kepa,0.0,2,False,,None,45,109745.jpg,3.4,False,Arrizabalaga Revuelta,1.0,False,None,a,1,3,106,0,0,0,0,0.0,23.6,Arrizabalaga,200.0,2025-07-01,1994-10-03,False,p109745,2790,0,0,8,39,0,0,0,3,0,97,7,406,744.4,10.4,4.0,75.8,0,0,0,0,31,0.06,0.03,0.09,38.83,38,9,344,12,333,1,183,9,NaN,,NaN,,NaN,,0.0,3.13,0.00,0.00,1.25,1.26,372,22,144,51,94,16,199,28,1.00,0.26,0.00
2,True,True,NaN,None,463748,0,0,0,0,0,1,1.5,None,0,Karl,0.0,3,False,,None,40,463748.jpg,0.0,False,Hein,0.6,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Hein,67.0,2020-10-28,2002-04-13,False,p463748,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,437,55,427,46,396,31,437,55,NaN,,NaN,,NaN,,0.0,0.00,0.00,0.00,0.00,0.00,610,63,58,29,440,55,247,33,0.00,0.00,0.00
3,True,True,NaN,None,551221,0,0,0,0,0,1,1.5,None,0,Tommy,0.0,4,False,,None,40,551221.jpg,0.0,False,Setford,0.2,False,None,a,1,3,0,0,0,0,0,0.0,0.0,Setford,241.0,2024-07-24,2006-03-13,False,p551221,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.00,0.00,0.00,0.00,471,66,462,57,436,44,471,66,NaN,,NaN,,NaN,,0.0,0.00,0.00,0.00,0.00,0.00,653,76,101,42,474,66,451,61,0.00,0.00,0.00
4,True,True,NaN,None,226597,0,0,0,0,0,2,4.5,None,0,Gabriel,0.0,5,False,,None,60,226597.jpg,4.2,False,dos Santos Magalhães,21.5,False,None,a,1,3,117,0,0,0,0,0.0,19.5,Gabriel,30.0,2020-09-01,1997-12-19,False,p226597,2363,3,2,10,22,0,0,0,4,0,0,9,459,584.6,208.8,287.0,108.2,0,0,0,159,28,2.70,1.21,3.91,23.09,96,32,167,36,107,12,121,31,NaN,,NaN,,NaN,,0.1,0.00,0.05,0.15,0.88,0.84,98,5,584,219,32,1,15,6,1.07,0.38,6.06


In [33]:
print(players.columns)

Index(['can_transact', 'can_select', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'code', 'cost_change_event',
       'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall',
       'dreamteam_count',
       ...
       'now_cost_rank_type', 'form_rank', 'form_rank_type',
       'points_per_game_rank', 'points_per_game_rank_type', 'selected_rank',
       'selected_rank_type', 'starts_per_90', 'clean_sheets_per_90',
       'defensive_contribution_per_90'],
      dtype='object', length=101)


In [34]:
# select columns of interest from players df

players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,David,Raya Martín,Raya,Arsenal,GKP
1,2,Kepa,Arrizabalaga Revuelta,Arrizabalaga,Arsenal,GKP
2,3,Karl,Hein,Hein,Arsenal,GKP
3,4,Tommy,Setford,Setford,Arsenal,GKP
4,5,Gabriel,dos Santos Magalhães,Gabriel,Arsenal,DEF


In [15]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)


  0%|          | 0/677 [00:00<?, ?it/s]

In [ ]:
points

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2021/22,154561,45,44,95,2160,0,0,8,27,0,0,0,1,0,78,5,496,593.4,10.0,0.0,60.1,0,0,0.00,0.00,0.00,0.00
1,2022/23,154561,45,48,166,3420,0,0,12,46,1,0,0,1,0,154,20,822,1146.0,20.1,4.0,117.3,0,38,0.11,0.12,0.23,50.12
2,2023/24,154561,50,53,135,2880,0,0,16,24,0,1,0,2,0,46,6,569,413.0,0.0,0.0,41.1,0,32,0.00,0.04,0.04,22.51
3,2024/25,154561,55,56,142,3420,0,0,13,34,0,0,0,3,0,86,10,555,755.4,10.7,0.0,76.7,0,38,0.00,0.03,0.03,35.03
0,2018/19,109745,55,54,142,3240,0,0,14,39,0,1,0,2,0,82,7,685,681.0,0.1,0.0,68.0,0,0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023/24,589507,45,45,5,173,0,0,0,4,0,0,0,0,0,0,0,12,7.8,1.2,13.0,2.2,0,2,0.10,0.02,0.12,1.89
1,2024/25,589507,45,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.00,0.00
0,2022/23,531363,45,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.00,0.00,0.00,0.00
1,2023/24,531363,45,45,7,183,0,0,1,3,0,0,0,1,0,0,0,17,22.4,29.2,36.0,8.6,0,1,0.32,0.06,0.38,3.25


In [ ]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

KeyError: 'element'